Main use cases using OpenAI functions for Tagging and Extraction 

This allows us to  extract structured data from unstructured text


Tagging:-

We have seen the LLM, given a function description, select arguments from the input text, generate a structured output forming a function call.

More generally, the LLM can evaluated the input text and generated structured output.


Text ---> LLM ----> {
           ^             Sentiment: Positive
           |             language: spanish
           |         }
      Structured
      Description


Here we pass in unstructured text along with some structured description, and then we use the LLM to generate some structured output to reason over that input text

and create some response response in the format of the structured description that we pass in.

Here in above eg, we know we have been generating an object that has a sentiment of the text and also has a tag for the language of the text.

So we pass in a piece of text we will pass in a structure description saying hey extract some sentiment, extract some language, and the LLM will 

reason over that text and respond with am object that has a sentiments tag and a language tag.



Extraction 

1. When given an input Json schema, the LLM gas been fine tuned to find and fill the parameters of that schema.

2. The capability is not limited to function schema.

3. This can be used for general purpose extraction.


Text ----> LLM ----> [{ ...
            ^            first_name: lang,
            |            last_name: chain,
            |            language: python
            |          },
            |         ]
        Structured 
        Description


In extraction, we are going to be extracting specific entities from the text. 

These entities are also represented by a structure description.

But rather than using LLM to reason over the text and respond with a single output in this structure description.

We are using LLM to look over the text and extract a list of these elements.

eg look over an article and extract the list of papers that are mentioned in articles.
        

In [1]:
import os
from dotenv import load_dotenv


In [2]:
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
from typing import List   # to help us with type hints
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [4]:
# Create pydantic model

class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment: str = Field(description = "Sentiment of text, should be `pos`, `neg` or `neutral`")
    language: str = Field(description = "Language of the text (should be ISO 639-1 code)")

In [5]:
convert_pydantic_to_openai_function(Tagging)

/var/folders/s8/qyjb36g92fs3ztdqk120mmkw0000gn/T/ipykernel_2244/1725091243.py:1: LangChainDeprecationWarning: The function `_convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  convert_pydantic_to_openai_function(Tagging)


{'name': 'Tagging',
 'description': 'Tag the piece of text with particular info.',
 'parameters': {'properties': {'sentiment': {'description': 'Sentiment of text, should be `pos`, `neg` or `neutral`',
    'type': 'string'},
   'language': {'description': 'Language of the text (should be ISO 639-1 code)',
    'type': 'string'}},
  'required': ['sentiment', 'language'],
  'type': 'object'}}

Here we got json block of function which is going to pass to openai as openai can only take functions in json format.

In [6]:
# Tagging

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

In [7]:
model = ChatOpenAI(temperature = 0)

/var/folders/s8/qyjb36g92fs3ztdqk120mmkw0000gn/T/ipykernel_2244/3583057845.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(temperature = 0)


In [8]:
tagging_functions = [convert_pydantic_to_openai_function(Tagging)]

In [11]:
prompt = ChatPromptTemplate.from_messages([
    ("system"," Think carefully, and them tag the text as instructed"),
    ("user","{input}")   
])


In [ ]:
model_with_functions = model.bind(
    functions = tagging_functions
    function_call = {"name":"Tagging"}
)

In [13]:
tagging_chain = prompt | model_with_functions

In [17]:
import json

In [19]:
# print(json.dumps(response.model_dump(),indent=2))
# json.dumps(tagging_chain.invoke({"input":"I love langchain"}).model_dump(),indent=2)
tagging_chain.invoke({"input":"I love langchain"})


AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"pos","language":"en"}', 'name': 'Tagging'}}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 101, 'total_tokens': 120, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--d7b134b6-8546-4577-8a23-953e42d4b0cd-0')

Here we can see the function call is there arguments that are passes in and we can see that the sentiment is positive and language is english

In [20]:
tagging_chain.invoke({"input":"non mi piace questo cibo"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"neg","language":"it"}', 'name': 'Tagging'}}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 104, 'total_tokens': 123, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--0c4506bc-e597-4701-835d-6a03eda437ec-0')

Here we can see the function call is there arguments that are passes in and we can see that the sentiment is negative and language is italian.

here we can see output format is kind of nested and we know that we are always going to be extracting the structure and so what we really want to do

is add an output parser that takes in this AI message and basically pares out the JSON and just says that because that's the only interesting thing here we already

know that we are going to call this function, so 

the fact that content is null, that not interesting to us.

the fact that there's a function call that's made, that not interesting to us.

we are forcing to do that.

The fact that it's calling tagging function, also not interesting to us because we know that it's going to call tagging function because we forced it to.

Here we what really want is just the values of arguments, which is JSON block and it will be really convenient if that was parsed into JSON because it's JSON

in this JSON block and we want to be able to use the individual elements.

we can use Output parser, in chain that would help us.


In [22]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [23]:
tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

In [24]:
tagging_chain.invoke({"input":"non mi piace questo cibo"})

{'sentiment': 'neg', 'language': 'it'}

here we only got parameters

Extraction

Extraction is similar to tagging, but used for extracting multiple pieces of information.

In [25]:
from typing import Optional
class Person(BaseModel):
    """Information about a person."""
    name: str = Field(description = "Person's name")
    age: Optional[int] = Field(description = "Person's age")

we want to extract list of person objects so

In [26]:
class Information(BaseModel):
    """Information to extract."""
    people: List[Person] = Field(description= "List of info about people")

This Information class will be pass to opneai function

In [27]:
convert_pydantic_to_openai_function(Information)

{'name': 'Information',
 'description': 'Information to extract.',
 'parameters': {'properties': {'people': {'description': 'List of info about people',
    'items': {'description': 'Information about a person.',
     'properties': {'name': {'description': "Person's name", 'type': 'string'},
      'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
       'description': "Person's age"}},
     'required': ['name', 'age'],
     'type': 'object'},
    'type': 'array'}},
  'required': ['people'],
  'type': 'object'}}

In [29]:
extraction_functions = [convert_pydantic_to_openai_function(Information)]
extraction_model = model.bind(functions= extraction_functions, function_call={"name":"Information"})

In [31]:
extraction_model.invoke("Joe is 30, his mom is martha")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"Joe","age":30},{"name":"Martha","age":null}]}', 'name': 'Information'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 96, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--66c89739-8567-48b3-9b4c-891eb5b22684-0')

Here we got all the parameters for the person, but considering martha, age is showing null, we can probably do better by forcing model to respond in more educated way.

By adding prompt tht will tell the Language model to do that.

In [54]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info if necessary if variable is optional"),
    ("human", "{input}")      
])

This will allow language model to not respond always to makeup age equals 0.

In [55]:
extraction_chain = prompt | extraction_model

In [56]:
extraction_chain.invoke({"input":"Joe is 30, his mom is martha"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"Joe","age":30},{"name":"Martha"}]}', 'name': 'Information'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 119, 'total_tokens': 136, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--46273007-5026-4ddb-b27d-4d298bc567b2-0')

we can parse this ai message to some structure

In [57]:
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()

In [58]:
extraction_chain.invoke({"input":"Joe is 30, his mom is martha"})


{'people': [{'name': 'Joe', 'age': 30}, {'name': 'Martha'}]}

we dont care about list of people mentioning above can use different output parser

JsonKeyOutputFunctionsParser:- will look for a particular key in the output and extract only that.

In [60]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [63]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name = 'people')

In [64]:
extraction_chain.invoke("Joe is 30, his mom is martha")

[{'name': 'Joe', 'age': 30}, {'name': 'Martha'}]

In [65]:
extraction_chain.invoke({"input":"Joe is 30, his mom is martha"})

[{'name': 'Joe', 'age': 30}, {'name': 'Martha'}]

Lets try with real world example.

In [69]:
# Real world example.

from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

In [71]:
doc = documents[0]

In [72]:
page_content=doc.page_content[:10000]

In [73]:
print(page_content)







LLM Powered Autonomous Agents | Lil'Log







































Lil'Log

















|






Posts




Archive




Search




Tags




FAQ









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.


In [78]:
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info if necessary if variable is optional"),
#     ("human", "{input}")      
# ])

In [79]:
# create class describing what we want to tag. 
#we want to get overview of this article and we want extract summary.

class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written in.")
    keywords: str = Field(description="Provide keywords related to the content.")

In [80]:
overview_tagging_function = [
    convert_pydantic_to_openai_function(Overview)
]

tagging_model= model.bind(
    functions=overview_tagging_function,
    function_call = {"name":"Overview"}
)

tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()


In [81]:
tagging_chain.invoke({"input":page_content})

{'summary': 'The article discusses building autonomous agents powered by LLM (large language model) as the core controller, with components like planning, memory, and tool use. It explores task decomposition, self-reflection, and challenges in implementing LLM-powered agents.',
 'language': 'English',
 'keywords': 'LLM, autonomous agents, planning, memory, tool use, task decomposition, self-reflection, challenges'}

In [83]:
# extract papers mentioned in the article

class Paper(BaseModel):
    """Information about papers mentioned."""
    title: str
    author: Optional[str]


class Info(BaseModel):
    """Information to extract"""
    papers: List[Paper]

In [85]:
paper_extraction_function = [
    convert_pydantic_to_openai_function(Info)
]

extraction_model = model.bind(
    functions = paper_extraction_function,
    function_call = {"name":"Info"}
)

extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name = "papers")

In [86]:
extraction_chain.invoke({"input":page_content})

[{'title': 'LLM Powered Autonomous Agents', 'author': 'Lilian Weng'}]

This is little bit confusing as this actually title of article we are passing in, and author of article that we are passing in.

So language model probably getting confuse because it remember the initial page is a lot of this article title, this is author and we 

havn't really instructed model clearly that it should be extracting title of papers which are mentioned within rather than the information about the article itself.

To fix that we hav to give slightly better system message.

In [87]:
template = """ A article will be passed to you. Extract from all papers that are mentioned by this article follow by its author. 

Do not extract the name of the article itself. If no papers are mentioned that is fine - you don't need to extract any! Just return an empty list.

Do not make up or guess Any extra information. Only extract what exactly is in the text."""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human","{input}")
])

In [88]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name = "papers")

In [89]:
extraction_chain.invoke({"input":page_content})

[{'title': 'Chain of thought (CoT; Wei et al. 2022)',
  'author': 'Wei et al. 2022'},
 {'title': 'Tree of Thoughts (Yao et al. 2023)', 'author': 'Yao et al. 2023'},
 {'title': 'LLM+P (Liu et al. 2023)', 'author': 'Liu et al. 2023'},
 {'title': 'ReAct (Yao et al. 2023)', 'author': 'Yao et al. 2023'},
 {'title': 'Reflexion (Shinn & Labash 2023)', 'author': 'Shinn & Labash 2023'},
 {'title': 'Chain of Hindsight (CoH; Liu et al. 2023)',
  'author': 'Liu et al. 2023'},
 {'title': 'Algorithm Distillation (AD; Laskin et al. 2023)',
  'author': 'Laskin et al. 2023'}]

We can do some sanity checks to check if its performing ok.

In [90]:
extraction_chain.invoke({"input":"hi"})

[]

Had done only only 1st 1000 words what if want to do on whole article.

In order to do that we have to use text splitting. as this article is really really long and so if we try to pass that article directly 

to LLM, its actually going to be too big for token window. and so what we are going to do is we are going to split it into smaller pieces of text and 

then we are going to pass those to the language model individually and then we are going to combine all the results at the end.

In [91]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)

In [92]:
splits = text_splitter.split_text(doc.page_content)

In [93]:
len(splits)

15

create a function that will join lists

because we will extract a list of papers mentioned in each split and then merge them all together.

Have some method of preparing the splits to be passed into the chain.

Chain takes in an input variable specifically you need a dictionary with a input key.

In [94]:
def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

In [96]:
flatten([[1,2,3],[4,5,6]])

[1, 2, 3, 4, 5, 6]

In [97]:
print(splits[0])

LLM Powered Autonomous Agents | Lil'Log







































Lil'Log

















|






Posts




Archive




Search




Tags




FAQ









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent 

here we can see 1st element in the list is consist of just text. so we gonna need some way of taking this list of text and converting it into a list of dictionaries.

where that text is now the input key.

# we are going to do that by defining function because this going to be 1st function in the chain we are gonna wrap it in a runnable lambda.

what is runnable lambda:- is just a simple wrapper in Langchain that takes in a function or lambda and then converts it into a runnable object.

It is necessary to do when your functions are the 1st elements in the chain so that they can be successfully pipe together.

In [98]:
from langchain.schema.runnable import RunnableLambda

In [99]:
prep = RunnableLambda(
    lambda x: [{"input":doc} for doc in text_splitter.split_text(x)]
)

In [100]:
prep.invoke("hi")

[{'input': 'hi'}]

This is important because it will going to be input for next extraction chain and so we want to create a bunch of inputs there.

Extraction_chain works on single element but we have list of inputs here so we can use map(), it basically takes previous input, ie list and map extraction_chain over them.

it will lead to list of lists. as extraction chain will be returning list. 

Then will merge the list using flatten function, we don't want to wrap it in Runnable lambda as its not a 1st function in the chain. although we can wrap it if want to but ite

not necessary.

In [101]:
chain = prep | extraction_chain.map() | flatten

In [ ]:
chain.invoke(doc.page_content)

Its first splitting it in 15 sections, then passing that to extraction_chain. when it passes to extraction chain, its actually parallelizing, lot of those 

calls automatically. the default that it parallelizes by is 5 calls., so its not full y parallelizing, but it's speeding up significant amount.

when all those calls are done, it will then get passed into the final flatten function.